In [ ]:
import pandas as pd 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import MeanShift, estimate_bandwidth
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [2]:
dt=pd.read_csv('results.csv')

In [3]:
dt.head(5)

,division,name,city,gender,age,official,bib,overall,state,genderdiv,net,country
0,4 / 26,"Soejima, Masazumi",Fukuoka City,M,41,83.45,W1,4 / 35,NaN,4 / 26,83.45,JPN
1,14 / 4335,"Gebremariam, Gebregziabher",Tigray,M,27,142.93,2,14 / 21616,NaN,14 / 12621,142.93,ETH
2,6 / 26,"Van Dyk, Ernst F.",Paarl,M,39,84.38,W2,6 / 35,NaN,6 / 26,84.38,RSA
3,2 / 26,"Fearnley, Kurt H.",Hamilton,M,31,81.65,W3,2 / 35,NaN,2 / 26,81.65,AUS
4,3 / 26,"Hokinoue, Kota",Iizuka,M,38,83.43,W4,3 / 35,NaN,3 / 26,83.43,JPN


the column 'division','overall','genderdiv'  is stand for the participant position in the race based on gender,kind of race.
from the information i found on the internet , bib is the id of participant based on kind of race.
W is for wheelchair. 
F is for female running.
and only number is male running.

In [4]:
dt.dtypes

division      object
name          object
city          object
gender        object
age            int64
official      object
bib           object
overall       object
state         object
genderdiv     object
net          float64
country       object
dtype: object

From the data . i see the time they finish marathon is 'official' column , but it in object format , so i will change it to number . 
i have my focus in 'age' and 'official' and i have a hypothesize that 'official' is different with group of 'age'. 

Change the 'official' type from object to number.

In [11]:
dt['official']=dt['official'].apply(pd.to_numeric, errors = 'coerce')

In [12]:
dt.describe()

,age,official,net
count,21652.000000,21576.000000,21652.00000
mean,41.836135,262.798014,258.64258
std,10.858308,50.383191,49.94535
min,18.000000,78.420000,78.42000
25%,33.000000,228.645000,224.63000
50%,42.000000,255.720000,251.07500
75%,50.000000,290.630000,285.53000
max,83.000000,475.320000,570.27000


From describe table we can clearly see that 75% of participant have age around 50 and the limit age to join the race is above 18 yrs, no upper limit for age <br>
And the count on official is different so i drop all drow have null value in official column. <br>

In [8]:
X.dropna(subset=['official'],inplace=True)

Now i will do and k-mean clustering between 'age' and 'official'
But i want to test the efficiency for different number of clusters , i will use an loop to run k-mean with different number of clusters and choose the best case based on the silhouette score.

In [7]:
X=pd.concat([dt['official'],dt['age']],axis=1)

### 1/ K-means clustering.

In [ ]:
clusters=[2,3,4,5,6,7,8]
a=X.values
for number in clusters:
    kmeans = KMeans(number, random_state=0)
    result = kmeans.fit_predict(a)
    silhouette_avg = silhouette_score(a, result)
    fig, (ax2) = plt.subplots(1)
    # 2nd Plot showing the actual clusters formed
    colors = cm.spectral(result.astype(float) / number)
    ax2.scatter(a[:, 0], a[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')
    # Labeling the clusters
    centers = kmeans.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')
    print("For n_clusters =", number,"The average silhouette_score is :", silhouette_avg)
    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')


For n_clusters = 2 The average silhouette_score is : 0.5376479153093278 <br>
For n_clusters = 3 The average silhouette_score is : 0.46776155873189174 <br>
For n_clusters = 4 The average silhouette_score is : 0.4320362358070203 <br>
For n_clusters = 5 The average silhouette_score is : 0.40735777139395873 <br>
For n_clusters = 6 The average silhouette_score is : 0.38544754017254446 <br>
For n_clusters = 7 The average silhouette_score is : 0.3662045066597228 <br>
For n_clusters = 8 The average silhouette_score is : 0.34492257895003064 <br>

From the result of silhouette_score we can see the best number of cluster for kmeans is 2.

### 2/ Mean Shift clustering.

In [ ]:
qt=[0.1,0.2]
for quant in qt:
    a=X.values
    bandwidth = estimate_bandwidth(a, quantile=quant)
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    result = ms.fit_predict(a)
    silhouette_avg = silhouette_score(a, result)
    print("For quantile =", quant,"The average silhouette_score is :", silhouette_avg)

For quantile = 0.1 The average silhouette_score is : 0.3747231056490832 <br>
For quantile = 0.2 The average silhouette_score is : 0.37381604716089645 <br>

For cluster using MeanShift , when i use quantile over 0.2 there are error on my model so i use only 0.1 and 0.2 for quantile , and the result show that for qualtile =0.1 the model are better. 
The result when compare with kmean is much lower.

### 3/ DBSCAN  Clustering.

Next,i will use the DBSCAN to cluster, with the change in eps parameter.

In [ ]:
a=X.values
ep=[0.1,0.2,0.3,0.4,0.5,0.7,0.8,0.9]
for i in ep:
    db = DBSCAN(eps=i, min_samples=5)
    result = db.fit_predict(a)
    silhouette_avg = silhouette_score(a, result)
    print("For eps=",i,"The average silhouette_score is :", silhouette_avg)

For eps= 0.1 The average silhouette_score is : -0.7454988600902304 <br>
For eps= 0.2 The average silhouette_score is : -0.5500833114826047 <br>
For eps= 0.3 The average silhouette_score is : -0.3732881664721746 <br>
For eps= 0.4 The average silhouette_score is : -0.2826773969367739 <br>
For eps= 0.5 The average silhouette_score is : -0.26260844237542424 <br>
For eps= 0.7 The average silhouette_score is : -0.3335863605040074 <br>
For eps= 0.8 The average silhouette_score is : -0.347475068082284 <br>
For eps= 0.9 The average silhouette_score is : -0.39630005091769194 <br>

the silhouette_score highest when eps=0.5, but in overall the score is still very low 

### 4/ Conclusion 

After using 3 method of clustering , i see the K-means have highest silhouette_score and take short time to calculate. <br>
For other methods like Affinity propagation and Spectral clustering , my colab notebook return runtime died so i cant show the result. <br>